In [1]:
import pandas as pd
from pandas_profiling import ProfileReport as pr
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import requests
import json
from sklearn.linear_model import LinearRegression
import numpy as np
from time import sleep
from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.ar_model import ar_select_order
from autots import AutoTS
tqdm.pandas()

In [2]:
train_data = pd.read_csv("Challenge1_train_data.csv")

In [3]:
train_data.head()

,date,dc_name,size_code,retail_price,total_tires,zip_code
0,2022-02-05,OAKLAND,1856015,70.090,1,94604
1,2021-03-02,OAKLAND,1856015,59.090,1,94604
2,2020-12-08,OAKLAND,1856015,53.545,1,94604
3,2022-01-17,OAKLAND,1856015,58.410,1,94604
4,2020-11-13,OAKLAND,1856015,43.085,1,94604


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284700 entries, 0 to 284699
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          284700 non-null  object 
 1   dc_name       284700 non-null  object 
 2   size_code     284700 non-null  int64  
 3   retail_price  284666 non-null  float64
 4   total_tires   284700 non-null  int64  
 5   zip_code      284700 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 13.0+ MB


In [5]:
train_data.date.min(),train_data.date.max()

('2020-09-20', '2022-09-19')

In [6]:
test_date = parse(train_data.date.max()) + relativedelta(days=-14)
str(test_date)

'2022-09-05 00:00:00'

In [7]:
forecast_df = train_data[train_data['date']>str(parse(train_data.date.max()) + relativedelta(days=-7))]
forecast_df['date'] = pd.to_datetime(forecast_df['date']) + timedelta(days=7)
forecast_df['date'] = forecast_df['date'].astype(str)
forecast_df = forecast_df[['date','dc_name','size_code']].sort_values(by=['dc_name','size_code','date']).reset_index().drop(columns='index')
forecast_df

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/713131145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_df['date'] = pd.to_datetime(forecast_df['date']) + timedelta(days=7)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/713131145.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_df['date'] = forecast_df['date'].astype(str)


,date,dc_name,size_code
0,2022-09-20,BAKERSFIELD,1856015
1,2022-09-21,BAKERSFIELD,1856015
2,2022-09-22,BAKERSFIELD,1856015
3,2022-09-23,BAKERSFIELD,1856015
4,2022-09-24,BAKERSFIELD,1856015
...,...,...,...
2725,2022-09-22,SAN JOSE,22570195
2726,2022-09-23,SAN JOSE,22570195
2727,2022-09-24,SAN JOSE,22570195
2728,2022-09-25,SAN JOSE,22570195


In [8]:
profile = pr(train_data, title="ATD Data Analysis", explorative=True)
profile.to_file("atd_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:4812: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
train_data[train_data['retail_price']==0]

,date,dc_name,size_code,retail_price,total_tires,zip_code
236727,2021-01-17,OAKLAND,1856015,0.0,0,94604
236728,2022-03-20,OAKLAND,1856015,0.0,0,94604
236729,2020-09-29,OAKLAND,1856015,0.0,0,94604
236730,2021-07-13,OAKLAND,1856015,0.0,0,94604
236731,2022-04-06,OAKLAND,1856015,0.0,0,94604
...,...,...,...,...,...,...
284695,2022-07-24,BAKERSFIELD,3512520,0.0,0,93308
284696,2020-11-22,BAKERSFIELD,3512520,0.0,0,93308
284697,2021-08-08,BAKERSFIELD,3512520,0.0,0,93308
284698,2022-02-14,BAKERSFIELD,3512520,0.0,0,93308


In [10]:
len(train_data)

284700

In [11]:
###here we are getting the weather data from the api into a dataframe for all the cities


weather_sacramento_url = f'https://archive-api.open-meteo.com/v1/era5?latitude=38.5816&longitude=-121.4944&start_date=2020-09-20&end_date=2022-09-19&timezone=GMT&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max' 
weather_bakersfield_url= f'https://archive-api.open-meteo.com/v1/era5?latitude=35.3733&longitude=-119.0187&start_date=2020-09-20&end_date=2022-09-19&timezone=GMT&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max'
weather_sanjose_url = f'https://archive-api.open-meteo.com/v1/era5?latitude=37.3387&longitude=-121.8853&start_date=2020-09-20&end_date=2022-09-19&timezone=GMT&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max'
weather_oakland_url = f'https://archive-api.open-meteo.com/v1/era5?latitude=37.8044&longitude=-122.2712&start_date=2020-09-20&end_date=2022-09-19&timezone=GMT&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max'


def get_weather_data(url,city_name):
    weather_resp = requests.get(url)
    a=weather_resp.json()
    weather=pd.DataFrame.from_dict(a.get('daily'))
    weather['city_name']=city_name
    return weather

weather_sacramento=get_weather_data(weather_sacramento_url,'SACRAMENTO')
print(weather_sacramento.info())

weather_bakersfield=get_weather_data(weather_bakersfield_url,'BAKERSFIELD')
print(weather_bakersfield.info())
weather_sanjose=get_weather_data(weather_sanjose_url,'SAN JOSE')
print(weather_sanjose.info())
weather_oakland= get_weather_data(weather_oakland_url,'OAKLAND')
print(weather_oakland.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   time                730 non-null    object 
 1   temperature_2m_max  730 non-null    float64
 2   temperature_2m_min  730 non-null    float64
 3   precipitation_sum   730 non-null    float64
 4   windspeed_10m_max   730 non-null    float64
 5   city_name           730 non-null    object 
dtypes: float64(4), object(2)
memory usage: 34.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   time                730 non-null    object 
 1   temperature_2m_max  730 non-null    float64
 2   temperature_2m_min  730 non-null    float64
 3   precipitation_sum   730 non-null    float64
 4   windspeed_10m_max   730 non-null    float64

In [12]:
weather_data = pd.concat([weather_sacramento, weather_bakersfield,weather_sanjose,weather_oakland],ignore_index=True)
weather_data.head()

,time,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,city_name
0,2020-09-20,33.3,15.3,0.0,8.5,SACRAMENTO
1,2020-09-21,33.8,16.4,0.0,10.7,SACRAMENTO
2,2020-09-22,32.3,17.0,0.0,12.3,SACRAMENTO
3,2020-09-23,32.7,15.5,0.0,11.8,SACRAMENTO
4,2020-09-24,33.1,17.3,0.0,12.5,SACRAMENTO


In [13]:
train_data = train_data.merge(weather_data, how='left', left_on=['date', 'dc_name'], right_on=['time', 'city_name']).drop(columns=['city_name','time'])
train_data.head()

,date,dc_name,size_code,retail_price,total_tires,zip_code,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max
0,2022-02-05,OAKLAND,1856015,70.090,1,94604,17.5,4.8,0.0,7.4
1,2021-03-02,OAKLAND,1856015,59.090,1,94604,20.3,6.3,0.0,12.9
2,2020-12-08,OAKLAND,1856015,53.545,1,94604,23.6,5.6,0.0,10.0
3,2022-01-17,OAKLAND,1856015,58.410,1,94604,15.8,6.8,0.0,7.4
4,2020-11-13,OAKLAND,1856015,43.085,1,94604,16.9,7.5,0.0,21.5


## Exploratory Data Anlaysis

In [14]:
zero_data = train_data[train_data['retail_price']==0]
grouped_size_data = zero_data.groupby(['dc_name','size_code'])['date'].count()
grouped_size_data

dc_name      size_code
BAKERSFIELD  1856015      128
             1856514      181
             1856515      118
             1956015      115
             1956515      115
                         ... 
SAN JOSE     2757018      159
             2854522      127
             2857017      207
             3512520       97
             22570195     182
Name: date, Length: 390, dtype: int64

In [15]:
grouped_dc_data = zero_data.groupby(['dc_name'])['date'].count()
grouped_dc_data

dc_name
BAKERSFIELD    13663
OAKLAND        11282
SACRAMENTO     15428
SAN JOSE        7600
Name: date, dtype: int64

## Data Classification

In [16]:
train_data['Key'] = train_data['dc_name'] +'|' +train_data['size_code'].astype(str)

In [17]:
train_data['date'] = pd.to_datetime(train_data['date'])
train_data.head()

,date,dc_name,size_code,retail_price,total_tires,zip_code,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,Key
0,2022-02-05,OAKLAND,1856015,70.090,1,94604,17.5,4.8,0.0,7.4,OAKLAND|1856015
1,2021-03-02,OAKLAND,1856015,59.090,1,94604,20.3,6.3,0.0,12.9,OAKLAND|1856015
2,2020-12-08,OAKLAND,1856015,53.545,1,94604,23.6,5.6,0.0,10.0,OAKLAND|1856015
3,2022-01-17,OAKLAND,1856015,58.410,1,94604,15.8,6.8,0.0,7.4,OAKLAND|1856015
4,2020-11-13,OAKLAND,1856015,43.085,1,94604,16.9,7.5,0.0,21.5,OAKLAND|1856015


In [18]:
train_df = train_data[train_data['date']<str(test_date)]

In [19]:
cv_data = train_df.groupby(['Key']).agg(average=('total_tires','mean'),
                                                    sd=('total_tires','std')).reset_index()
#cv_data['key_adi']= cv_data['dc_name']+'|'+cv_data['size_code'].astype(str)

In [20]:
## Calculating CV_squared

cv_data['cv_sqr'] = (cv_data['sd']/cv_data['average'])**2
cv_data

,Key,average,sd,cv_sqr
0,BAKERSFIELD|1856015,9.195804,7.457122,0.657602
1,BAKERSFIELD|1856514,5.411189,5.668122,1.097218
2,BAKERSFIELD|1856515,12.304895,10.371903,0.710495
3,BAKERSFIELD|1956015,28.994406,20.119600,0.481515
4,BAKERSFIELD|1956515,34.744056,23.725029,0.466286
...,...,...,...,...
385,SAN JOSE|2756520,5.565035,5.771771,1.075678
386,SAN JOSE|2757018,7.676923,6.810600,0.787039
387,SAN JOSE|2854522,7.345455,6.092618,0.687972
388,SAN JOSE|2857017,6.467133,6.428880,0.988205


In [21]:
#train_df['key_adi'] = train_df['dc_name']+'|'+train_df['size_code'].astype(str)

non_zero_df = train_df[train_df['total_tires']>0]
non_zero_df = non_zero_df.dropna(subset=('total_tires'))
tire_by_date= non_zero_df.groupby(['Key','date']).agg(count=('Key','count')).reset_index()

In [22]:
skus=tire_by_date.Key.value_counts()
skus

SACRAMENTO|1956515    702
SACRAMENTO|2155517    701
SACRAMENTO|2055516    696
SACRAMENTO|2156016    695
SACRAMENTO|2256517    693
                     ... 
SAN JOSE|2056016      491
SACRAMENTO|2356016    489
SACRAMENTO|2154018    489
SAN JOSE|2555020      487
SACRAMENTO|11225      483
Name: Key, Length: 390, dtype: int64

In [23]:
new_df= pd.DataFrame()
for i in range(len(skus.index)):
    a= tire_by_date[tire_by_date['Key']==skus.index[i]]
    a['previous_date']=a['date'].shift(1)
    new_df=pd.concat([new_df,a],axis=0)

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

...

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_49942/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [24]:
new_df['duration']=new_df['date']- new_df['previous_date']

In [25]:
new_df.head()

,Key,date,count,previous_date,duration
117189,SACRAMENTO|1956515,2020-09-20,1,NaT,NaT
117190,SACRAMENTO|1956515,2020-09-21,1,2020-09-20,1 days
117191,SACRAMENTO|1956515,2020-09-22,1,2020-09-21,1 days
117192,SACRAMENTO|1956515,2020-09-23,1,2020-09-22,1 days
117193,SACRAMENTO|1956515,2020-09-24,1,2020-09-23,1 days


In [26]:
new_df['Duration']=new_df['duration'].astype(str).str.replace('days','')
new_df['Duration']=pd.to_numeric(new_df['Duration'],errors='coerce')
ADI = new_df.groupby('Key').agg(ADI = ('Duration','mean')).reset_index()
ADI

,Key,ADI
0,BAKERSFIELD|1856015,1.206780
1,BAKERSFIELD|1856514,1.328358
2,BAKERSFIELD|1856515,1.186667
3,BAKERSFIELD|1956015,1.182724
4,BAKERSFIELD|1956515,1.182724
...,...,...
385,SAN JOSE|2756520,1.390625
386,SAN JOSE|2757018,1.275986
387,SAN JOSE|2854522,1.206430
388,SAN JOSE|2857017,1.385214


In [27]:
adi_cv=pd.merge(ADI,cv_data)
adi_cv

,Key,ADI,average,sd,cv_sqr
0,BAKERSFIELD|1856015,1.206780,9.195804,7.457122,0.657602
1,BAKERSFIELD|1856514,1.328358,5.411189,5.668122,1.097218
2,BAKERSFIELD|1856515,1.186667,12.304895,10.371903,0.710495
3,BAKERSFIELD|1956015,1.182724,28.994406,20.119600,0.481515
4,BAKERSFIELD|1956515,1.182724,34.744056,23.725029,0.466286
...,...,...,...,...,...
385,SAN JOSE|2756520,1.390625,5.565035,5.771771,1.075678
386,SAN JOSE|2757018,1.275986,7.676923,6.810600,0.787039
387,SAN JOSE|2854522,1.206430,7.345455,6.092618,0.687972
388,SAN JOSE|2857017,1.385214,6.467133,6.428880,0.988205


In [28]:
def category(df):
    a=0
    
    if((df['ADI']<=1.34) & (df['cv_sqr']<=0.49)):
        a='Smooth'
    if((df['ADI']>=1.34) & (df['cv_sqr']>=0.49)):  
        a='Lumpy'
    if((df['ADI']<1.34) & (df['cv_sqr']>0.49)):
        a='Erratic'
    if((df['ADI']>1.34) & (df['cv_sqr']<0.49)):
        a='Intermittent'
    return a

In [29]:
adi_cv['category']=adi_cv.apply(category,axis=1)
adi_cv[['Key','category']]

,Key,category
0,BAKERSFIELD|1856015,Erratic
1,BAKERSFIELD|1856514,Erratic
2,BAKERSFIELD|1856515,Erratic
3,BAKERSFIELD|1956015,Smooth
4,BAKERSFIELD|1956515,Smooth
...,...,...
385,SAN JOSE|2756520,Lumpy
386,SAN JOSE|2757018,Erratic
387,SAN JOSE|2854522,Erratic
388,SAN JOSE|2857017,Lumpy


In [30]:
adi_cv.category.value_counts()


Erratic    246
Smooth      98
Lumpy       46
Name: category, dtype: int64

In [31]:
train_data = train_data.merge(adi_cv[['Key','category']],on=['Key'],how='left')

In [32]:
train_data

,date,dc_name,size_code,retail_price,total_tires,zip_code,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,Key,category
0,2022-02-05,OAKLAND,1856015,70.090,1,94604,17.5,4.8,0.0,7.4,OAKLAND|1856015,Lumpy
1,2021-03-02,OAKLAND,1856015,59.090,1,94604,20.3,6.3,0.0,12.9,OAKLAND|1856015,Lumpy
2,2020-12-08,OAKLAND,1856015,53.545,1,94604,23.6,5.6,0.0,10.0,OAKLAND|1856015,Lumpy
3,2022-01-17,OAKLAND,1856015,58.410,1,94604,15.8,6.8,0.0,7.4,OAKLAND|1856015,Lumpy
4,2020-11-13,OAKLAND,1856015,43.085,1,94604,16.9,7.5,0.0,21.5,OAKLAND|1856015,Lumpy
...,...,...,...,...,...,...,...,...,...,...,...,...
284695,2022-07-24,BAKERSFIELD,3512520,0.000,0,93308,36.0,21.1,0.0,8.0,BAKERSFIELD|3512520,Erratic
284696,2020-11-22,BAKERSFIELD,3512520,0.000,0,93308,17.7,4.6,0.0,9.4,BAKERSFIELD|3512520,Erratic
284697,2021-08-08,BAKERSFIELD,3512520,0.000,0,93308,35.9,21.4,0.0,8.7,BAKERSFIELD|3512520,Erratic
284698,2022-02-14,BAKERSFIELD,3512520,0.000,0,93308,23.9,6.8,0.0,9.7,BAKERSFIELD|3512520,Erratic


### Handling Missing Values

In [33]:
train_data[train_data['retail_price'].isna()]

,date,dc_name,size_code,retail_price,total_tires,zip_code,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,Key,category
77863,2020-12-03,SACRAMENTO,1757014,NaN,4,95838,18.4,1.8,0.0,8.1,SACRAMENTO|1757014,Erratic
81565,2022-04-04,SACRAMENTO,1857014,NaN,2,95838,28.6,11.6,0.0,18.2,SACRAMENTO|1857014,Erratic
82169,2021-09-17,SACRAMENTO,1955515,NaN,1,95838,29.2,12.3,0.0,13.6,SACRAMENTO|1955515,Lumpy
84743,2020-11-12,SACRAMENTO,1957014,NaN,2,95838,17.3,2.3,0.0,5.4,SACRAMENTO|1957014,Erratic
93686,2022-04-04,SACRAMENTO,2154018,NaN,4,95838,28.6,11.6,0.0,18.2,SACRAMENTO|2154018,Lumpy
97510,2020-10-23,SACRAMENTO,2155518,NaN,3,95838,26.3,12.2,0.0,12.3,SACRAMENTO|2155518,Erratic
97587,2021-11-01,SACRAMENTO,2155518,NaN,4,95838,19.0,13.7,3.8,10.0,SACRAMENTO|2155518,Erratic
98654,2020-11-16,SACRAMENTO,2156017,NaN,2,95838,21.0,6.9,0.0,6.5,SACRAMENTO|2156017,Erratic
100598,2022-05-23,SACRAMENTO,2157015,NaN,2,95838,36.5,17.1,0.0,14.0,SACRAMENTO|2157015,Erratic
102646,2020-12-11,SACRAMENTO,2158516,NaN,4,95838,18.8,4.4,0.0,10.9,SACRAMENTO|2158516,Erratic


In [34]:
# Impute the missing values for the retail_price
df_sacramento = train_data[train_data['dc_name']=='SACRAMENTO']

sac_sizecode = df_sacramento[df_sacramento['retail_price'].isna()]['size_code'].tolist()

for code in sac_sizecode : 
    # Get the mean value
    mean_value = df_sacramento[(df_sacramento['size_code'] == code) & ((~df_sacramento['retail_price'].isna()) & (df_sacramento['retail_price'] != 0))]['retail_price'].mean()
    df_sacramento.loc[(df_sacramento['size_code'] == code) & (df_sacramento['retail_price'].isna()), 'retail_price'] = mean_value

In [35]:
# Putting the sacramento data into the main dataframe after imputing
indexes_to_drop = train_data[train_data['dc_name']=='SACRAMENTO'].index
train_data.drop(index= indexes_to_drop, inplace = True)
train_data = pd.concat([
    train_data,df_sacramento
], ignore_index= True)

In [36]:
train_data['day_name'] = train_data['date'].dt.day_name()

In [37]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284700 entries, 0 to 284699
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date                284700 non-null  datetime64[ns]
 1   dc_name             284700 non-null  object        
 2   size_code           284700 non-null  int64         
 3   retail_price        284700 non-null  float64       
 4   total_tires         284700 non-null  int64         
 5   zip_code            284700 non-null  int64         
 6   temperature_2m_max  284700 non-null  float64       
 7   temperature_2m_min  284700 non-null  float64       
 8   precipitation_sum   284700 non-null  float64       
 9   windspeed_10m_max   284700 non-null  float64       
 10  Key                 284700 non-null  object        
 11  category            284700 non-null  object        
 12  day_name            284700 non-null  object        
dtypes: datetime64[ns](1), float64

In [38]:
train_data.to_csv("cleansed_data.csv",index=False)

In [39]:
forecast_df.to_csv("forecast_data.csv",index=False)

## Univariate Models

In [40]:
uni_train_data = train_data[['Key','dc_name','size_code','date','category','total_tires']].sort_values(['Key','dc_name','size_code','date'])
#uni_train_data = uni_train_data[uni_train_data['category'].isin(['Smooth','Erratic'])].drop(columns='category')
uni_train_data = uni_train_data.drop(columns='category')
uni_train_data

,Key,dc_name,size_code,date,total_tires
157968,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-20,0
76544,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-21,18
76183,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-22,7
76064,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-23,3
76485,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-24,15
...,...,...,...,...,...
75002,SAN JOSE|3512520,SAN JOSE,3512520,2022-09-15,8
75018,SAN JOSE|3512520,SAN JOSE,3512520,2022-09-16,10
157638,SAN JOSE|3512520,SAN JOSE,3512520,2022-09-17,0
157665,SAN JOSE|3512520,SAN JOSE,3512520,2022-09-18,0


In [41]:
uni_train_data['date'] = pd.to_datetime(uni_train_data['date'])
uni_train_data.head()

,Key,dc_name,size_code,date,total_tires
157968,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-20,0
76544,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-21,18
76183,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-22,7
76064,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-23,3
76485,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-24,15


#### Linear Regression

In [42]:
def linear_reg(sliced_data,future_df):
    #train_df = uni_train_data[uni_train_data['date']<str(test_date)][['date','total_tires']]
    #test_df = uni_train_data[uni_train_data['date']>=str(test_date)][['date','total_tires']]
    key = uni_train_data['Key'].unique()[0]
    train_df = uni_train_data[['date','total_tires']]
    future_df['Key'] = future_df['dc_name'] +'|' +future_df['size_code'].astype(str)
    test_df = future_df[future_df['Key']==key]
    test_df = test_df[['date']]
    test_df['date'] = pd.to_datetime(test_df['date'])
    train_df=train_df.reset_index()
    test_df=test_df.reset_index()
    x_train = train_df['date'].values.reshape(-1, 1)
    x_train = x_train.astype(float)
    y_train = train_df['total_tires'].values
    
    x_test = test_df['date'].values.reshape(-1, 1)
    x_test = x_test.astype(float)
    
    
    
    
    regressor = LinearRegression() 
        
    regressor.fit(x_train,y_train)
        

    y_test = regressor.predict(x_test)
    test_df['Forecast'] = y_test.round()
    test_df['Model'] = 'Linear_Regression'
    test_df['Type'] = 'Univariate'
    return test_df
    

#### Light GBM

In [43]:
import lightgbm as lgb
def light_gbm(sliced_data):
    train_df = uni_train_data[uni_train_data['date']<str(test_date)][['date','total_tires']]
    test_df = uni_train_data[uni_train_data['date']>=str(test_date)][['date','total_tires']]
    
    train_df=train_df.reset_index()
    test_df=test_df.reset_index()
    x_train = train_df['date'].values.reshape(-1, 1)
    x_train = x_train.astype(float)
    y_train = train_df['total_tires'].values
    
    x_test = test_df['date'].values.reshape(-1, 1)
    x_test = x_test.astype(float)
    t_test = test_df['total_tires'].values
    
    
    
    iteration = 15000
                                                                                                       
    lgb_params = {                                                                                                                            
        'nthread': -1,
        'metric': 'mae',
        'boosting_type': 'gbdt',    
        'max_depth': 7,
        'num_leaves': 28,   
        'task': 'train',                                                                                                                      
        'objective': 'regression_l1',                                                                                                         
        'learning_rate': 0.05,                                                                                                                
        'feature_fraction': 0.9,                                                                                                              
        'bagging_fraction': 0.8,                                                                                                              
        'bagging_freq': 5,                                                                                                                    
        'lambda_l1': 0.06,                                                                                                                    
        'lambda_l2': 0.05,                                                                                                                    
        'verbose': -1,     }                                                                                                                           
                                                                                                                                              
    # LightGBM dataset                                                                                                                        
    lgbtrain_all = lgb.Dataset(data=x_train, label=y_train)                                                                
    final_model = lgb.train(lgb_params, lgbtrain_all, num_boost_round=iteration)                                                              
    test_preds = final_model.predict(x_test, num_iteration=iteration)
    test_df['Forecast'] = test_preds.round()
    test_df['Model'] = 'Light_BGM'
    test_df['Type'] = 'Univariate'
    return test_df
    


In [44]:
linear_output = uni_train_data.groupby('Key').apply(linear_reg,forecast_df).reset_index()
linear_output = linear_output[['Key','date','Forecast','Model']]
print(len(linear_output))
linear_output = linear_output.merge(uni_train_data[['Key','dc_name','size_code']].drop_duplicates(),on='Key',how='left')
print(len(linear_output))

linear_output

2730
2730


,Key,date,Forecast,Model,dc_name,size_code
0,BAKERSFIELD|1856015,2022-09-20,16.0,Linear_Regression,BAKERSFIELD,1856015
1,BAKERSFIELD|1856015,2022-09-21,16.0,Linear_Regression,BAKERSFIELD,1856015
2,BAKERSFIELD|1856015,2022-09-22,16.0,Linear_Regression,BAKERSFIELD,1856015
3,BAKERSFIELD|1856015,2022-09-23,16.0,Linear_Regression,BAKERSFIELD,1856015
4,BAKERSFIELD|1856015,2022-09-24,16.0,Linear_Regression,BAKERSFIELD,1856015
...,...,...,...,...,...,...
2725,SAN JOSE|3512520,2022-09-22,16.0,Linear_Regression,SAN JOSE,3512520
2726,SAN JOSE|3512520,2022-09-23,16.0,Linear_Regression,SAN JOSE,3512520
2727,SAN JOSE|3512520,2022-09-24,16.0,Linear_Regression,SAN JOSE,3512520
2728,SAN JOSE|3512520,2022-09-25,16.0,Linear_Regression,SAN JOSE,3512520


In [ ]:
lgbm_output = uni_train_data.groupby('Key').apply(light_gbm).reset_index()
lgbm_output = lgbm_output[['Key','date','total_tires','Forecast','Model']]

lgbm_output

In [45]:
forecast_data = forecast_df.copy()
forecast_data['Key'] = forecast_data['dc_name'] + '|'+forecast_data['size_code'].astype(str)
forecast_data.head()

,date,dc_name,size_code,Key
0,2022-09-20,BAKERSFIELD,1856015,BAKERSFIELD|1856015
1,2022-09-21,BAKERSFIELD,1856015,BAKERSFIELD|1856015
2,2022-09-22,BAKERSFIELD,1856015,BAKERSFIELD|1856015
3,2022-09-23,BAKERSFIELD,1856015,BAKERSFIELD|1856015
4,2022-09-24,BAKERSFIELD,1856015,BAKERSFIELD|1856015


In [47]:
train_data = train_data[['date','Key','total_tires']]
train_data['date'] = pd.to_datetime(train_data['date'])
forecast_data['date'] = pd.to_datetime(forecast_data['date'])

#### ARIMA

In [48]:
def arima(sliced_df):
    global forecast_data
    future_data = forecast_data.copy()
    sliced_df.set_index('date', inplace=True)
    sliced_df = sliced_df.sort_index(ascending=True)
    key = sliced_df['Key'].unique()[0]
    #future_data = forecast_data.copy()

    test_df = future_data[future_data['Key'] == key]
    
    auto_arima = pm.auto_arima(sliced_df[['total_tires']], stepwise=False, seasonal=False)
    #model=ARIMA(sliced_df['total_tires'],order=(1,1,1))
    #model_fit=model.fit()
    y_test = auto_arima.predict(n_periods=7)
    test_df['Forecast'] = y_test.tolist()
    test_df['Model'] = 'ARIMA'
    test_df['Type'] = 'Univariate'
    test_df = test_df.drop(columns='Key')
    return test_df


In [ ]:
arima_output = train_data.groupby('Key').progress_apply(arima).reset_index()

In [ ]:
arima_output = arima_output[['date','dc_name','size_code','Forecast']].rename(columns={'Forecast':'total_tires'})
arima_output['date'] = arima_output['date'].astype(str)
arima_output

#### Auto Regressor

In [49]:
def auto_reg(sliced_df):
    global forecast_data
    key = sliced_df['Key'].unique()[0]
    future_data = forecast_data.copy()
    sliced_df.set_index('date', inplace=True)
    sliced_df = sliced_df.sort_index(ascending=True)
    
    

    test_df = future_data[future_data['Key'] == key]
    mod = ar_select_order(sliced_df[['total_tires']], maxlag=13)
    
    ar_model = AutoReg(sliced_df[['total_tires']],lags=7,seasonal=True).fit()
    
    y_test = ar_model.forecast(7)
    test_df['Forecast'] = y_test.tolist()
    test_df['Model'] = 'Auto_Reg'
    test_df['Type'] = 'Univariate'
    
    required_df = test_df[['date','dc_name','size_code','Forecast']]
    return required_df


In [50]:
auto_reg_output = train_data.groupby('Key').progress_apply(auto_reg).reset_index()

  0%|                                                   | 0/390 [00:00<?, ?it/s]/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending.

/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:435: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpoth

/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:435: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
/Users/akhileshpoth

/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:435: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency 

/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast

...

 94%|██████████████████████████████████████▎  | 365/390 [00:06<00:00, 53.78it/s]/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending.

 95%|███████████████████████████████████████  | 371/390 [00:06<00:00, 54.93it/s]/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending.

 97%|███████████████████████████████████████▋ | 377/390 [00:07<00:00, 55.36it/s]/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending.

 98%|████████████████████████████████████████▎| 383/390 [00:07<00:00, 54.50it/s]/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending.

100%|████████████████████████████████████████▉| 389/390 [00:07<00:00, 52.26it/s]/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending.

In [51]:
auto_reg_output = auto_reg_output[['date','dc_name','size_code','Forecast']].rename(columns={'Forecast':'total_tires'})
auto_reg_output['total_tires'] = auto_reg_output['total_tires'].round().astype(int)
auto_reg_output['date'] = auto_reg_output['date'].astype(str)
auto_reg_output

,date,dc_name,size_code,total_tires
0,2022-09-20,BAKERSFIELD,1856015,9
1,2022-09-21,BAKERSFIELD,1856015,9
2,2022-09-22,BAKERSFIELD,1856015,10
3,2022-09-23,BAKERSFIELD,1856015,11
4,2022-09-24,BAKERSFIELD,1856015,12
...,...,...,...,...
2725,2022-09-22,SAN JOSE,3512520,13
2726,2022-09-23,SAN JOSE,3512520,16
2727,2022-09-24,SAN JOSE,3512520,10
2728,2022-09-25,SAN JOSE,3512520,2


#### AutoML

In [52]:
from autots.models.model_list import model_lists
print(model_lists.keys())

dict_keys(['all', 'default', 'fast', 'superfast', 'parallel', 'fast_parallel', 'probabilistic', 'multivariate', 'univariate', 'no_params', 'recombination_approved', 'no_shared', 'no_shared_fast', 'experimental', 'slow', 'gpu', 'regressor', 'best', 'motifs'])


In [53]:
print(model_lists['univariate'])

['UnivariateRegression', 'UnobservedComponents', 'ConstantNaive', 'Greykite', 'NeuralProphet', 'ETS', 'ARIMA', 'Theta', 'GLS', 'ARDL', 'FBProphet', 'ARCH', 'SeasonalNaive', 'AverageValueNaive', 'LastValueNaive', 'GLM', 'DatepartRegression', 'UnivariateMotif']


In [54]:
def auto_ml(sliced_data,future_data):
    model = AutoTS(
    forecast_length=7,
    frequency='D',
    prediction_interval=0.95,
    ensemble=None,
    models_mode='deep',
    model_list = 'univariate',# or ['ARIMA','ETS']
    max_generations=1,
    num_validations=3,
    no_negatives=True,
    n_jobs='auto')

    key = sliced_data['Key'].unique()[0]

    test_df = future_data[future_data['Key'] == key]
    
    sliced_data.set_index('date', inplace=True)
    model.fit(sliced_data['total_tires'])
    prediction = model.predict(forecast_length=7)
    
    test_df['Forecast'] = prediction.forecast['total_tires'].values.tolist()
    test_df['Forecast'] = test_df['Forecast'].round()
    test_df['Model'] = 'Auto_ML_Best_Model'
    test_df['Type'] = 'Univariate'
    test_df = test_df.drop(columns='Key')
    return test_df

In [ ]:
auto_output = train_data.groupby('Key').apply(auto_ml,forecast_data).reset_index()

In [ ]:
auto_output = auto_output[['date','dc_name','size_code','Forecast']].rename(columns={'Forecast':'total_tires'})
auto_output

#### Best Model

In [55]:
df_submission = auto_reg_output

In [56]:
'''import requests
url = "https://scoring-app-uuzeqpiufa-ue.a.run.app/forecast/submit"
payload = {
  
"team_key": "9MaRyrU5",
"data": df_submission.to_dict(orient="records")}
response = requests.post(url, json=payload) 
print(response.status_code, response.content)
'''

200 b'{"message":"Success"}'
